<a href="https://colab.research.google.com/github/CodeMonkey01/DataMining1/blob/main/SVM_Scherb_Floris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
df = files.upload()


Saving IMDB Dataset.csv to IMDB Dataset (2).csv


In [ ]:
import pandas as pd

df = pd.read_csv('IMDB Dataset.csv')
df.describe(include='all')

df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.iloc[3,0]

"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them."

In [ ]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
%pip install tensorflow-text
import pandas as pd
from sklearn.preprocessing import LabelEncoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **Getting an overview over the Data:**

In [ ]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [ ]:
df[df["sentiment"]== "positive"]

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
...,...,...
49983,"I loved it, having been a fan of the original ...",positive
49985,Imaginary Heroes is clearly the best film of t...,positive
49989,I got this one a few weeks ago and love it! It...,positive
49992,John Garfield plays a Marine who is blinded by...,positive


In [ ]:
df[df["sentiment"]== "negative"]

,review,sentiment
3,Basically there's a family where a little boy ...,negative
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
10,Phil the Alien is one of those quirky films wh...,negative
11,I saw this movie when I was about 12 when it c...,negative
...,...,...
49994,This is your typical junk comedy.<br /><br />T...,negative
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


##**Check for imbalance**

In [ ]:
df_pos = df[df["sentiment"]== "positive"]
df_neg = df[df["sentiment"]== "negative"]

#drop the difference between the positive and negative set.

df_raw_pos = df.iloc[186: , : ]
df_raw = pd.concat([df_raw_pos,df], ignore_index=True)
df_raw.sample(frac=1).reset_index(drop=True)

#Check for balance now.

print(df_raw["sentiment"].value_counts())

positive    49922
negative    49892
Name: sentiment, dtype: int64


# **Clean the reviews from HTML Tags**

Example:

In [ ]:
df.iloc[16569,0]

'They say that it is always better in horror movies to leave things to the imagination of the viewer- to hide certain details from the audience in order to tickle their sense of imagination, dip into their fears and let that give birth to their darkest thoughts.<br /><br />That was not the case when I watched Bakjwi, under the American title Thirst. Now playing at select theaters near you. Seems like the film makers did not want to spare you any details. There WILL be blood in this film and you WILL try to look away.<br /><br />For rest of review please visit http://without-terebi.blogspot.com/2009/08/thirst-aka-bakjwi.html Thanks and hope you enjoyed reading above.'

In [ ]:
#16569
#view the reviews with HTML tags in the sentences.
#pd.set_option('display.max_colwidth', None)

df[df['review'].str.contains(r'<[^<>]*>') == True]

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49994,This is your typical junk comedy.<br /><br />T...,negative
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative


In [ ]:
#remove HTML Tags

df['review'] = df['review'].str.replace(r'<[^<>]*>', '', regex=True)

In [ ]:
# todo --> take (random) sample to speed up training

df_sampled = df.sample(20_000)
df = df
print(df_sampled["sentiment"].value_counts())

positive    10041
negative     9959
Name: sentiment, dtype: int64


In [ ]:
print(df["sentiment"].value_counts())

positive    25000
negative    25000
Name: sentiment, dtype: int64


HTML Tags successfully removed

In [ ]:
df.iloc[16569,0]

'They say that it is always better in horror movies to leave things to the imagination of the viewer- to hide certain details from the audience in order to tickle their sense of imagination, dip into their fears and let that give birth to their darkest thoughts.That was not the case when I watched Bakjwi, under the American title Thirst. Now playing at select theaters near you. Seems like the film makers did not want to spare you any details. There WILL be blood in this film and you WILL try to look away.For rest of review please visit http://without-terebi.blogspot.com/2009/08/thirst-aka-bakjwi.html Thanks and hope you enjoyed reading above.'

# **Preprocessing**

As wrote in the paper, we tried out different preprocessing approaches for BERT.

Stop word removal
Stemming
Seperator / Special tokens
We tried these preprocessing independently and together. After testing each method (or combined with others) we found out that option 3 ("Seperator / Special tokens) is working the best for BERT and this binary classification problem. The code for the other preprocessing methods is listed below.

In [ ]:
ex = df_raw.iloc[1:2]

In [ ]:
ex

,review,sentiment
1,While I count myself as a fan of the Babylon 5...,negative


In [ ]:
ex["sentiment"] = ex["sentiment"].apply(lambda x: 1 if x == "positive" else 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# Transform class from Boolean to integer value.

df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x == "positive" else 0)
# df_sampled['sentiment'].apply(lambda x: 1 if x == "positive" else 0)
# df['sentiment'].apply(lambda x: 1 if x=="positive" else 0)

# **Stop word removal**

In [ ]:
# Remove stop words

# from gensim.parsing.preprocessing import remove_stopwords

# df['stop_word']=df['review'].apply(lambda x: remove_stopwords(x))

# **Stemming**

In [ ]:
#import re
#import nltk
#from nltk.stem import PorterStemmer

#token_pattern = re.compile(r"(?u)\b\w\w+\b")

#ps = PorterStemmer()

#nltk.download('punkt')
#nltk.download('stopwords')

#df['stemmed']=df['review'].apply(lambda x: ' '.join([ps.stem(y) for y in token_pattern.findall(x)]))

# **Stemming + Stop word removal**

In [ ]:
# Stemming + stop word

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re

token_pattern = re.compile(r"(?u)\b\w\w+\b")

ps = PorterStemmer()

nltk.download('punkt')
nltk.download('stopwords')

my_stopwords = set(stopwords.words('english'))

df['stemmed_stop_removed']=df['review'].apply(lambda x: ' '.join([ps.stem(y) for y in token_pattern.findall(x) if y not in my_stopwords]))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Stemming and stopword removal works

In [ ]:
df.iloc[16569,2]

'they say alway better horror movi leav thing imagin viewer hide certain detail audienc order tickl sens imagin dip fear let give birth darkest thought that case watch bakjwi american titl thirst now play select theater near seem like film maker want spare detail there will blood film will tri look away for rest review pleas visit http without terebi blogspot com 2009 08 thirst aka bakjwi html thank hope enjoy read'

# **SVM**



In [ ]:
df.head(5)

,review,sentiment,stemmed_stop_removed
0,One of the other reviewers has mentioned that ...,1,one review mention watch oz episod hook they r...
1,A wonderful little production. The filming tec...,1,wonder littl product the film techniqu unassum...
2,I thought this was a wonderful way to spend ti...,1,thought wonder way spend time hot summer weeke...
3,Basically there's a family where a little boy ...,0,basic famili littl boy jake think zombi closet...
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,petter mattei love time money visual stun film...


**Define Target variable and redefine data for training**

In [ ]:
df_target = df['sentiment']
df_data = df.copy()
df_data = df_data.drop(columns='review')
df_data = df_data.rename({'sentiment': 'sentiment_variable'}, axis = 1)

df_data.head(5)

,sentiment_variable,stemmed_stop_removed
0,1,one review mention watch oz episod hook they r...
1,1,wonder littl product the film techniqu unassum...
2,1,thought wonder way spend time hot summer weeke...
3,0,basic famili littl boy jake think zombi closet...
4,1,petter mattei love time money visual stun film...


In [ ]:
df_target.head()

0    1
1    1
2    1
3    0
4    1
Name: sentiment, dtype: int64

Vectorize stemmed_stop_removed column

In [ ]:
# Tfidf vectorizer

from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
vectorizer = TfidfVectorizer(min_df=0.01)
matrix = vectorizer.fit_transform(df_data['stemmed_stop_removed'])
df_data = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names())


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#pd.set_option('max_columns', None)
df_data.head(5)

,10,100,11,12,15,1950,1970,1980,20,30,...,yeah,year,yet,york,you,young,younger,youth,zero,zombi
0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
1,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.088642,0.000000,0.0,0.0,0.0,0.000000
2,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.077966,0.0,0.000000,0.000000,0.096533,0.0,0.0,0.0,0.000000
3,0.09171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.316565
4,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.115406,0.000000,0.000000,0.0,0.0,0.0,0.000000


In [ ]:
df_data['sentiment']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
49995    0.0
49996    0.0
49997    0.0
49998    0.0
49999    0.0
Name: sentiment, Length: 50000, dtype: float64

## Here we train the final model with the identified parameters and calculate the scores accuracy afterwards.

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create train/test split
df_data_train, df_data_test, df_target_train, df_target_test = train_test_split(
    df_data, df_target, test_size=0.2, random_state=42)

#df_data_train.describe()
#df_data_test.describe()
#df_target_train.describe()
#df_target_test.describe()

#SVM classifier with parameters
svm = LinearSVC(random_state=42, tol=0.1,C=1,dual=True,penalty='l2', loss='hinge',max_iter=5000)

#train final model
svm.fit(df_data_train, df_target_train)

#test final model
df_prediction = svm.predict(df_data_test)

print("Accuracy: {}".format(accuracy_score(df_target_test, df_prediction)))

Accuracy: 0.8771


In [ ]:
df_data_train.head()

,10,100,11,12,15,1950,1970,1980,20,30,...,yeah,year,yet,york,you,young,younger,youth,zero,zombi
39087,0.058723,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30893,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45278,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.19552,0.0,...,0.0,0.111982,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16398,0.000000,0.0,0.0,0.0,0.0,0.0,0.168669,0.0,0.00000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13653,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_data_test.head()

,10,100,11,12,15,1950,1970,1980,20,30,...,yeah,year,yet,york,you,young,younger,youth,zero,zombi
33553,0.074037,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
9427,0.000000,0.0,0.0,0.100343,0.0,0.0,0.0,0.0,0.0,0.086895,...,0.0,0.048588,0.062952,0.0,0.0,0.0,0.0,0.0,0.100871,0.0
199,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
12447,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.115474,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
39489,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.048362,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [ ]:
df_target_train.head()

39087    0
30893    0
45278    1
16398    0
13653    0
Name: sentiment, dtype: int64

In [ ]:
df_target_test.head()

33553    1
9427     1
199      0
12447    1
39489    0
Name: sentiment, dtype: int64

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

#also try out different evaluation measures to make sure there are no differences
print("Precision: {}".format(precision_score(df_target_test,df_prediction)))
print("Recall: {}".format(recall_score(df_target_test,df_prediction)))
print("Precision: {}".format(f1_score(df_target_test,df_prediction)))

Precision: 0.8670520231213873
Recall: 0.8930343322087716
Precision: 0.8798514028741812
